# 출처: https://www.philschmid.de/gptq-llama

# optimum과 GPTQ로 오픈 LLM 양자화

허깅 페이스 Optimum 팀은 AutoGPTQ 라이브러리를 통해 언어 모델에 GPTQ 양자화를 적용하는 간단한 API를 제공했습니다. GPTQ 양자화를 활용하면 오픈 LLM을 8비트, 4비트, 3비트, 심지어 2비트까지 압축할 수 있어 성능 저하 없이 더 작은 하드웨어에서 실행할 수 있습니다.

이 블로그에서는 다음과 같은 내용을 배울 수 있습니다.

1. 개발 환경 설정
2. 양자화 데이터 세트 준비
3. 모델 적재 및 양자화
4. 성능 및 추론 속도 테스트
5. 보너스: 텍스트 생성 추론으로 추론 실행

시작하기 전에 GPTQ가 무엇인지 간단히 살펴보겠습니다.

_참고: 이 튜토리얼은 엔비디아 A10G GPU를 포함한 g5.2xlarge AWS EC2 인스턴스에서 작성 및 실행되었습니다._


## GPTQ란 무엇인가요?

[GPTQ](https://arxiv.org/abs/2210.17323)는 GPT 같은 대형 언어 모델(LLM)을 압축하는 사후 학습 양자화 기법입니다. GPTQ는 모델의 각 가중치를 저장하는 데 필요한 비트 수를 32비트에서 3~4비트로 줄여 GPT 모델을 압축합니다. 이렇게 하면 모델이 훨씬 적은 메모리를 차지하므로, 13B Llama2 모델도 단일 GPU와 같은 적은 하드웨어에서 실행할 수 있습니다. GPTQ는 각 레이어를 개별적으로 분석해 가중치를 근사화하여 전체 정확도를 유지합니다.

주요 장점은 다음과 같습니다.
* 모델의 가중치를 레이어별로 16비트 대신 4비트로 양자화해 필요한 메모리를 4배 줄입니다.
* 양자화는 점진적으로 이루어져 정확도 손실을 최소화합니다.
* fp16 모델과 동일한 지연 시간을 유지하면서 메모리 사용량은 4배 줄고, 사용자 정의 커널을 사용하면 더 빠를 수도 있습니다.(예: [Exllama](https://github.com/turboderp/exllama))
* 양자화된 모델 가중치는 디스크에 미리 저장할 수 있어 추론 시 양자화를 다시 수행하지 않고 바로 사용할 수 있습니다.

_참고: 현재 GPTQ 양자화는 텍스트 모델에만 적용됩니다. 또한 양자화 과정은 시간이 많이 걸릴 수 있습니다. 활용하려는 모델의 GPTQ 양자화 버전이 있는지 [허깅 페이스 허브](https://huggingface.co/models?search=gptq)에서 확인해보기 바랍니다._

---

## 1. 개발 환경 설정

코딩을 시작하기 전에, 먼저 필요한 의존성 패키지들을 설치합니다.

In [2]:
!pip install "torch==2.0.1" "transformers==4.32.1" "optimum==1.12.0" "auto-gptq==0.4.2" "accelerate==0.22.0" "safetensors>=0.3.1" --upgrade

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 2. 양자화 데이터 세트 준비

GPTQ는 사후 학습 양자화 기법이므로 모델을 양자화하기 위해 데이터 세트를 준비해야 합니다. [허깅 페이스 허브](https://huggingface.co/datasets)의 데이터 세트나 자신의 데이터 세트를 활용할 수 있습니다. 이 블로그에서는 허깅 페이스 허브의 [WikiText](https://huggingface.co/datasets/wikitext) 데이터 세트를 활용할 것입니다. 이 데이터 세트는 성능 손실을 최소화하면서 가중치를 양자화하는 데 사용됩니다. 양자화 데이터 세트는 최소 128개의 샘플을 포함하는 것이 좋습니다.

_참고: [TheBloke](https://huggingface.co/TheBloke)라는 매우 활발한 커뮤니티 멤버가 허깅페이스 허브에 수백 개의 GPTQ 가중치를 기여했습니다. 그는 주로 일반 도메인 모델의 양자화 데이터 세트로 wikitext를 활용합니다._

자신의 미세조정 데이터 세트를 양자화에 활용하고 싶다면 "id" 대신 리스트로 제공할 수 있습니다. 자세한 사항은 이 [예시](https://colab.research.google.com/drive/1_TIrmuKOFhuRRiTWN94iLKUFu6ZX4ceb)에서 확인할 수 있습니다.

In [3]:
# 허깅 페이스에 있는 데이터 세트 id

dataset_id = "wikitext2"

## 3. 모델 로드 및 양자화

Optimum은 `optimum.qptq` 네임스페이스에 `GPTQQuantizer`를 사용해 GPTQ 양자화를 통합합니다. 이 양자화기는 데이터 세트(아이디 또는 리스트), 비트 수, 모델 시퀀스 길이(model_seqlen)를 입력으로 받습니다. 추가 커스터마이징이 필요하면, 자세한 사항은 [여기](https://github.com/huggingface/optimum/blob/234a427450a7dcc978b227fa627ebcdab1764318/optimum/gptq/quantizer.py#L76)에서 확인할 수 있습니다.

In [4]:
from optimum.gptq import GPTQQuantizer

# GPTQ 양자화기
quantizer = GPTQQuantizer(bits=4, dataset=dataset_id, model_seqlen=4096)
quantizer.quant_method = "gptq"

양자화기를 생성한 후 트랜스포머를 활용해 모델을 로드할 수 있습니다. 이 예시에서는 다른 블로그 포스트 ["확장 가이드: Llama 2 인스트럭션 튜닝"](https://www.philschmid.de/instruction-tune-llama-2)에서 학습한 [Llama 2 7B](https://huggingface.co/philschmid/llama-2-7b-instruction-generator) 모델을 양자화합니다. GPTQ는 가중치를 int4로 양자화하고 활성화는 float16으로 유지하는 혼합 int4/fp16 양자화 방식을 채택하므로, 모델을 `fp16`으로 로드합니다.

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 허깅 페이스 모델 id
model_checkpoint = "NousResearch/Llama-2-7b-hf" # 게이트 아님
#model_checkpoint = "meta/llama-2-7b" # 게이트

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False) # 빠른 토크나이저의 버그
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, low_cpu_mem_usage=True, torch_dtype=torch.float16) # 우리는 모델을 의도적으로 fp16으로 로드합니다.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


모델을 로드하면 양자화할 준비가 완료됩니다.
_참고: 양자화 과정은 하드웨어에 따라 많은 시간이 소요될 수 있습니다. 이 예시에서는 A10G GPU 한 대에서 7B 모델의 양자화하는데 약 몇 분정도 걸렸습니다._

In [6]:
import os 
import json

# 모델을 양자화합니다.
quantized_model = quantizer.quantize_model(model, tokenizer)

# 양자화된 모델을 디스크에 저장합니다.
save_folder = "quantized_llama"
model.save_pretrained(save_folder, safe_serialization=True)

# 새롭고 빠른 토크나이저를 로드하고 디스크에 저장합니다.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint).save_pretrained(save_folder)

# TGI를 위한 quantize_config.json을 저장합니다.
with open(os.path.join(save_folder, "quantize_config.json"), "w", encoding="utf-8") as f:
  quantizer.disable_exllama = False
  json.dump(quantizer.to_dict(), f, indent=2)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Found modules on cpu/disk. Using Exllama backend requires all the modules to be on GPU. Setting `disable_exllama=True`
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:508: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


모델이 부분적으로 오프로드 됐으므로 오류를 피하기 위해 `disable_exllama`를 `True`로 설정합니다. 추론 및 운영 환경에 로드할 때는 Exllama 커널을 활용하기 위해 `config.json`을 변경해야 합니다.

In [7]:
with open(os.path.join(save_folder, "config.json"), "r", encoding="utf-8") as f:
  config = json.load(f)
  config["quantization_config"]["disable_exllama"] = False
  with open(os.path.join(save_folder, "config.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)

## 4. 성능 및 추론 속도 테스트

최신 버전의 트랜스포머에서는 `AutoModelForCausalLM` 클래스를 활용해 GPTQ 양자화된 모델을 직접 로드할 수 있습니다. 허깅 페이스에서 이미 양자화된 모델(예: [TheBloke/Llama-2-13B-chat-GPTQ](https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ))을 로드하거나, 직접 양자화한 모델을 로드할 수 있습니다. 여기에서는 양자화 결과를 테스트하기 위해 양자화된 모델을 디스크에서 로드하고, 양자화되지 않은 모델과 비교할 것입니다.

먼저 양자화되지 않은 모델을 로드하고 간단한 프롬프트로 테스트하겠습니다.

In [8]:
import time 

# 프롬프트는 모델의 미세조정을 기반으로 합니다.: https://www.philschmid.de/instruction-tune-llama-2#4-test-model-and-run-inference
# prompt = """### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# Dear [boss name],

# I'm writing to request next week, August 1st through August 4th,
# off as paid time off.

# I have some personal matters to attend to that week that require
# me to be out of the office. I wanted to give you as much advance
# notice as possible so you can plan accordingly while I am away.

# Thank you, [Your name]

# ### Response:
# """

prompt = """### Instruction:
What are some common ways to deploy a model on AWS?

### Response:
"""


# 텍스트를 생성하고 지연 시간을 측정하는 헬퍼 함수
def generate_helper(pipeline,prompt=prompt):
    # 워밍업
    for i in range(5):
      _ = pipeline("Warm up")

    # 간단한 방법으로 지연 시간을 측정합니다.
    start = time.time()
    out = pipeline(prompt, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)
    end = time.time()

    generated_text = out[0]["generated_text"][len(prompt):]

    latency_per_token_in_ms = ((end-start)/len(pipeline.tokenizer(generated_text)["input_ids"]))*1000

    # 생성된 텍스트와 지연 시간을 반환합니다.
    return {"text": out[0]["generated_text"][len(prompt):], "latency": f"{round(latency_per_token_in_ms,2)}ms/token"}


우리는 기본 트랜스포머 모델을 로드하고 `pipeline` 클래스를 활용해 추론을 실행할 수 있습니다.

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 허깅 페이스 모델 id
model_checkpoint = "NousResearch/Llama-2-7b-hf" # non-gated
#model_checkpoint = "meta/llama-2-7b" # gated

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map="auto", torch_dtype=torch.float16)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


기준값 생성

In [10]:
import torch 

vanilla_res = generate_helper(pipe)

print(f"Latency: {vanilla_res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Instruction: {vanilla_res['text']}")

# Latency: 37.49ms/token
# GPU memory: 12.62 GB
# Generated Instruction: Write a request for PTO letter to my boss

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Latency: 41.59ms/token
GPU memory: 1.63 GB
Generated Instruction: 
There are several options for deploying models in AWS, including:

* Using a managed service such as Amazon SageMaker Model Monitoring or Amazon SageMaker Model Deployer. These services can help automate the deployment process and ensure that your models are always up-to-date.
* Using a serverless function such as Amazon SageMaker Serverless Inference. This option can be a good choice if you want to deploy a model in a


In [1]:
# 정리
del pipe
del model 
del tokenizer
torch.cuda.empty_cache()

NameError: name 'pipe' is not defined

이제 기준값과 동일한 프롬프트로 양자화된 모델을 테스트할 수 있습니다. 기준값이 마련됐으니 GPTQ 양자화된 가중치를 테스트하고 검증할 수 있습니다. 이를 위해 새로운 `AutoModelForCausalLM` 클래스의 `gptq` 통합 기능을 활용해 `gptq` 가중치를 직접 로드할 수 있습니다.

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# GPTQ 가중치 경로
quantized_model_checkpoint = "quantized_llama"

q_tokenizer = AutoTokenizer.from_pretrained(quantized_model_checkpoint)
q_model = AutoModelForCausalLM.from_pretrained(quantized_model_checkpoint, device_map="auto", torch_dtype=torch.float16)

qtq_pipe = pipeline("text-generation", model=q_model, tokenizer=q_tokenizer)

이제 기준값과 동일한 프롬프트로 양자화된 모델을 테스트할 수 있습니다.

In [13]:
gpq_res = generate_helper(qtq_pipe)

print(f"Latency: {gpq_res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Instruction: {gpq_res['text']}")

# Latency: 36.0ms/token
# GPU memory: 3.83 GB
# Generated Instruction: Write a letter requesting time off

Latency: 38.16ms/token
GPU memory: 0.44 GB
Generated Instruction: 
There are several ways to deploy a model on AWS, depending on the requirements of the project and the available resources.

One popular way to deploy a model on AWS is to use Amazon SageMaker, which provides a range of services and tools for building, training, and deploying machine learning models. SageMaker allows you to use a range of pre-built algorithms, custom algorithms, and open source frameworks to train your models, and then deploy them to a range of


비교해 보면, 기본 모델은 약 12.6GB의 메모리가 필요했고 GPTQ 모델은 약 3.8GB의 메모리가 필요했으며, 성능은 동일했습니다. GPTQ를 활용하면 메모리를 약 4배 절약할 수 있습니다(참고: PyTorch에는 기본 커널이 있습니다).

텍스트 생성 추론을 활용하면 토큰당 약 `22.942983ms`의 지연 시간을 달성할 수 있으며, 이는 트랜스포머보다 2배 빠릅니다. 모델을 운영 환경에 배포할 계획이라면 텍스트 생성 추론을 활용하는 것을 추천합니다.